In [6]:
#grab the required packages
#%pip install pandas
%pip install FRB

from fred import Fred
#https://frb.readthedocs.io/en/latest/usage.html#id1

import pandas as pd
import numpy as np
import os
import json

#TODO: create JSON HOF to wrap every call in the conversion to json observations =  json.dumps(json.loads(fr.series.observations('MORTGAGE30US'))['observations'])

#FRED API KEY LOAD
API_KEY = os.environ.get('FRED_API_KEY')

#INIT THE CLIENT
fr = Fred(api_key=API_KEY,response_type='json')



Note: you may need to restart the kernel to use updated packages.


In [2]:
#parent level category
res = fr.category.details(97)

#list all metrics available in the housing category
print(fr.category.children(97))

#housing starts category
print(fr.category.details(32302))


#list all series available in this category
res = json.loads(fr.category.series(32302))
#target the seriess object and select housing starts (the first in the list)
print(res["seriess"][0])



{"categories":[{"id":32300,"name":"Housing Units Authorized by Building Permits","parent_id":97},{"id":32301,"name":"Housing Units Authorized, But Not Yet Started","parent_id":97},{"id":32302,"name":"Housing Starts","parent_id":97},{"id":32303,"name":"Housing Units Under Construction","parent_id":97},{"id":32304,"name":"Housing Units Completed","parent_id":97},{"id":32426,"name":"New Residential Sales","parent_id":97},{"id":33949,"name":"Manufactured Housing","parent_id":97}]}
{"categories":[{"id":32302,"name":"Housing Starts","parent_id":97}]}
{'id': 'HOUST', 'realtime_start': '2023-01-29', 'realtime_end': '2023-01-29', 'title': 'New Privately-Owned Housing Units Started: Total Units', 'observation_start': '1959-01-01', 'observation_end': '2022-12-01', 'frequency': 'Monthly', 'frequency_short': 'M', 'units': 'Thousands of Units', 'units_short': 'Thous. of Units', 'seasonal_adjustment': 'Seasonally Adjusted Annual Rate', 'seasonal_adjustment_short': 'SAAR', 'last_updated': '2023-01-19 

In [48]:
#call the historical housing starts series
print(json.loads(fr.series.details('HOUST'))['seriess'][0]['title'])
print(pd.date_range("1959/01/01", "2023/01/01"))



New Privately-Owned Housing Units Started: Total Units
DatetimeIndex(['1959-01-01', '1959-01-02', '1959-01-03', '1959-01-04',
               '1959-01-05', '1959-01-06', '1959-01-07', '1959-01-08',
               '1959-01-09', '1959-01-10',
               ...
               '2022-12-23', '2022-12-24', '2022-12-25', '2022-12-26',
               '2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30',
               '2022-12-31', '2023-01-01'],
              dtype='datetime64[ns]', length=23377, freq='D')


In [5]:
observations = json.dumps(json.loads(fr.series.observations('HOUST'))['observations'])
HOUST_DF = pd.read_json(observations)
#convert date strings to proper datetimes
HOUST_DF['date'] = pd.to_datetime(HOUST_DF['date'])
HOUST_DF.columns = ['realtime_start', 'realtime_end', 'date', 'HOUSING_STARTS']
HOUST_DF.drop(axis=1, columns=['realtime_start','realtime_end'], inplace=True)
HOUST_DF.set_index("date", inplace=True)

print(HOUST_DF.index.min())

#setting up a frame based on the max/min dates and merging in other data
index_df = pd.DataFrame(np.nan, index=pd.date_range("1959/01/01", "2023/01/01"), columns=['A', 'B'])

MAIN_FRAME = index_df.join(HOUST_DF)


#HOUST_DF = HOUST_DF.resample('D').ffill() #this forward fills the previous value up until a new value exists

#HOUST_DF.index = HOUST_DF.index.strftime('%Y/%m/%d')

print(MAIN_FRAME.head())


NameError: name 'fr' is not defined

In [27]:

print(json.loads(observations)['seriess'])
#pd.date_range()

[{'id': 'MSIM2', 'realtime_start': '2023-01-29', 'realtime_end': '2023-01-29', 'title': 'Monetary Services Index: M2 (preferred)', 'observation_start': '1967-01-01', 'observation_end': '2013-12-01', 'frequency': 'Monthly', 'frequency_short': 'M', 'units': 'Billions of Dollars', 'units_short': 'Bil. of $', 'seasonal_adjustment': 'Seasonally Adjusted', 'seasonal_adjustment_short': 'SA', 'last_updated': '2014-01-17 07:16:44-06', 'popularity': 25, 'group_popularity': 25, 'notes': 'The MSI measure the flow of monetary services received each period by households and firms from their holdings of monetary assets (levels of the indexes are sometimes referred to as Divisia monetary aggregates).\nPreferred benchmark rate equals 100 basis points plus the largest rate in the set of rates.\nAlternative benchmark rate equals the larger of the preferred benchmark rate and the Baa corporate bond yield.\nMore information about the new MSI can be found at\nhttp://research.stlouisfed.org/msi/index.html.'}

In [51]:
#call historical mortgage rates
observations = json.dumps(json.loads(fr.series.observations('MORTGAGE30US'))['observations'])
MTG_DF = pd.read_json(observations) 
MTG_DF.columns = ['realtime_start', 'realtime_end', 'date', 'MTG_RATE']

#convert date strings to proper datetimes
MTG_DF['date'] = pd.to_datetime(MTG_DF['date'])
MTG_DF.columns = ['realtime_start', 'realtime_end', 'date', 'MTG_RATE']
MTG_DF.drop(axis=1, columns=['realtime_start','realtime_end'], inplace=True)
MTG_DF.set_index('date', inplace=True)
# using the resample method
# https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.fillna.html
MTG_DF = MTG_DF.resample('D').ffill() #this forward fills the previous value up until a new value exists

#MTG_DF = MTG_DF.resample('M').ffill() #monthly downsample
print(MTG_DF.head())

            MTG_RATE
date                
1971-04-02      7.33
1971-04-03      7.33
1971-04-04      7.33
1971-04-05      7.33
1971-04-06      7.33


In [18]:
print(observations)
#merge dataframes
MAIN_FRAME = pd.merge(MTG_DF, HOUST_DF, left_index=True, right_index=True)
#TODO: reindexed data should use max and min of all retrieved datasets
print(MAIN_FRAME)

{"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "order_by": "search_rank", "sort_order": "desc", "count": 33, "offset": 0, "limit": 50, "seriess": [{"id": "MSIM2", "realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "title": "Monetary Services Index: M2 (preferred)", "observation_start": "1967-01-01", "observation_end": "2013-12-01", "frequency": "Monthly", "frequency_short": "M", "units": "Billions of Dollars", "units_short": "Bil. of $", "seasonal_adjustment": "Seasonally Adjusted", "seasonal_adjustment_short": "SA", "last_updated": "2014-01-17 07:16:44-06", "popularity": 25, "group_popularity": 25, "notes": "The MSI measure the flow of monetary services received each period by households and firms from their holdings of monetary assets (levels of the indexes are sometimes referred to as Divisia monetary aggregates).\nPreferred benchmark rate equals 100 basis points plus the largest rate in the set of rates.\nAlternative benchmark rate equals the larger of th

In [16]:
##### Begin Noah


from fred import Fred
# https://frb.readthedocs.io/en/latest/usage.html#id1

import pandas as pd
import os
import json

# FRED API KEY LOAD
API_KEY = os.environ.get("FRED_API_KEY")

# INIT THE CLIENT
fr = Fred(api_key=API_KEY, response_type="json")

# Example API Payload - will need to modify to accept list of tickers from browser. Backend API should probably be POST method
Data = (
    '{"series":{"HOUST":"interpolate","MORTGAGE30US":"interpolate"},"outputFreq":"D"}'
)

###predownloaded data for flight



API_PAYLOAD = json.loads(Data)

#NR Grab FRED series details function

##JT  this is really to determine the default frequency of the requested dataset - interesting
def grab_Series_Details(id):
    details = json.loads(fr.series.details(id))["seriess"][0]["frequency_short"]
    
    #details = fr.series.details(id)
    #details = json.loads(details)
    # Return series frequency
    frequency = details["seriess"][0]["frequency_short"]
    return frequency


# Grab FRED series data function
def grab_Series(id):

    #TODO build up a proper object for the series, use that to moved the dataset, freuqency, descriptive data around the system
    # then pass that object to the pandas stuff

    series_frequency = grab_Series_Details(id)

    # call data
    observations = json.dumps(json.loads(fr.series.observations(id))["observations"])

    SERIES_DF = pd.read_json(observations)

    # convert date strings to proper datetimes
    # Convert to end of month/quarter/year if applicable. Need to standardize to end of period for transformations later
    if series_frequency == "M":
        SERIES_DF["date"] = pd.to_datetime(SERIES_DF["date"]) + pd.offsets.MonthEnd()
    elif series_frequency == "Q":
        SERIES_DF["date"] = pd.to_datetime(SERIES_DF["date"]) + pd.offsets.QuarterEnd()
    elif series_frequency == "A":
        SERIES_DF["date"] = pd.to_datetime(SERIES_DF["date"]) + pd.offsets.YearEnd()
    else:
        SERIES_DF["date"] = pd.to_datetime(SERIES_DF["date"])

    # rename value column to id name
    SERIES_DF.rename(columns={"value": id}, inplace=True)
    SERIES_DF.drop(axis=1, columns=["realtime_start", "realtime_end"], inplace=True)
    SERIES_DF.set_index("date", inplace=True)

    return SERIES_DF


# Data transform functions

# fill
def transform_fill(series, transform_target):
    outputSeries = series.resample(transform_target).bfill()
    return outputSeries


# prorate
#TODO understand prorate - no clue
def transform_prorate(series, transform_target, series_base_freq):
    # Resample data to target frequency
    newDF = series
    if series_base_freq == "M":
        newDF["newdate"] = newDF.index + pd.offsets.MonthEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.MonthEnd(-1)
    elif series_base_freq == "Q":
        newDF["newdate"] = newDF.index + pd.offsets.QuarterEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.QuarterEnd(-1)
    elif series_base_freq == "A":

        newDF["newdate"] = newDF.index + pd.offsets.YearEnd(0) #TODO: read up on pandas offset function
        newDF["prevdate"] = newDF.newdate.shift(1) #TODO unsure what shift does
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.YearEnd(-1)

    else:  # Assume week
        newDF["newdate"] = newDF.index
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.Week(-1)

    newDF["divisor"] = (newDF["newdate"] - newDF["prevdate"]).dt.days
    outputSeries = newDF.resample(transform_target).bfill()
    outputSeries["oldVal"] = pd.to_numeric(outputSeries.iloc[:, 0], errors="coerce")

    outputSeries.iloc[:, 0] = outputSeries["oldVal"] / outputSeries["divisor"]
    outputSeries.drop(
        axis=1, columns=["newdate", "divisor", "oldVal", "prevdate"], inplace=True
    )
    return outputSeries


# interpolate
def transform_interpolate(series, transform_target):
    outputSeries = series.resample(transform_target).interpolate()
    # Currently using linear interpolation, but may want to consider using a spline - good stats question
    return outputSeries


# average
def transform_average(series, transform_target):
    outputSeries = series.resample(transform_target).mean()
    return outputSeries


# sum
def transform_sum(series, transform_target):
    outputSeries = series.resample(transform_target).sum()
    return outputSeries



#JT we receive a payload with a set of series to retrieve, and the frequency we want to transform them into
# we pull the frequency from it (only 1 shared on both datasets)
# loop thru provided tickers
#'{"series":{"HOUST":"interpolate","MORTGAGE30US":"interpolate"},"outputFreq":"D"}'

#we then form the dictionary we'll be adding our datasets to
# API_SERIES_DICT = API_PAYLOAD["series"]
# results in API_SERIES_DICT = {"HOUST":"interpolate","MORTGAGE30US":"interpolate"}

# set the target output frequency for all datasets
    # then u loop thru each series in the dict
    #  grabbing the key from the dict
    # sending that ID to the fred retrieval function
    # also sending the fill method for that series (e.g. interpolate, sum, average)

# 3 vars get passed to produce the master output
    # desired data freuqnecy
    # name of each series
    # fill method for each series

# noah then retieves each dataset, and adjusts to end of period at retireval time
# e.g. end of month, quarter, year  
# TODO: do we want to make this it's own function? might be too assumptive

# TODO var naming is very confusing

# Data processing master function

# For each series in paylaod
outputSeriesList = list()
outputSeriesTarget = API_PAYLOAD["outputFreq"]
API_SERIES_DICT = API_PAYLOAD["series"]

for payload in API_SERIES_DICT:
    #     Grab id and transform
    series_id = payload
    series_transform = API_SERIES_DICT[payload]
    
    #     Grab the series data (raw from FRED)
    SERIES_DATA = grab_Series(series_id)
    #     If the transform method requires series frequency
    #     Transform the series data as specified
    if series_transform == "fill":
        SERIES_DATA_TRANSFORM = transform_fill(SERIES_DATA, outputSeriesTarget)
    elif series_transform == "prorate":
        SERIES_DATA_FREQ = grab_Series_Details(series_id) #TODO: double API call here, we already get this data in the grab_Series function
        SERIES_DATA_TRANSFORM = transform_prorate(SERIES_DATA, outputSeriesTarget, SERIES_DATA_FREQ)
    elif series_transform == "interpolate":
        SERIES_DATA_TRANSFORM = transform_interpolate(SERIES_DATA, outputSeriesTarget)
    elif series_transform == "average":
        SERIES_DATA_TRANSFORM = transform_average(SERIES_DATA, outputSeriesTarget)
    elif series_transform == "sum":
        SERIES_DATA_TRANSFORM = transform_sum(SERIES_DATA, outputSeriesTarget)
    else:
        SERIES_DATA_TRANSFORM = SERIES_DATA
    # Store series in output list
    outputSeriesList.append(SERIES_DATA_TRANSFORM)

# Insert series merge code here:


# Convert the merged series to json and then return payload to browser here

##### End Noah Code ^^

[{"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-01-01", "value": "1657"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-02-01", "value": "1667"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-03-01", "value": "1620"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-04-01", "value": "1590"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-05-01", "value": "1498"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-06-01", "value": "1503"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-07-01", "value": "1547"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-08-01", "value": "1430"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "1959-09-01", "value": "1540"}, {"realtime_start": "2023-01-29", "realtime_end": "2023-01-29", "date": "

In [ ]:
################################################################################################################################################
################################################################# TJOKER #######################################################################
################################################################################################################################################

#TODO: break up this mammoth function :(

#receive payload (series names + fill prefs, target frequency)
@app.route("/giveMeData", methods=['POST'])
def data_request_hof():

    #read in the target frequency    
    target_output_frequency = request.json['target_frequency']
    
    #expect data_series to be a list of objects
    requested_series_identifier_list  = request.json['requested_series_identifier_list']

    #define list of objects to send to the next step
    outgoing_dataseries_list = []
    earliest_date = '1900/01/01'
    latest_date = '2023/01/28'

    #looooooooooooop
    for requested_series in requested_series_identifier_list:

        #2 params on each series object
        series_identifier = requested_series['series_identifier']
        fill_methodology = requested_series['fill_methodology']

        #define outgoing data  #TODO: future state hold on to the name and description from the original search result
        detailed_series_data = {
            'series_name': 'placeholder',
            'series_identifier': series_identifier,
            'series_description': 'placeholder',
            'series_raw_api_response': 'placeholder',
            'series_raw_observations': 'placeholder',
            'series_dataframed': 'placeholder',
            'series_fill_methodology': fill_methodology
            }

        #get the raw data from FRED
        detailed_series_data['series_raw_api_response'] = retrieve_raw_fred_data(series_identifier)

        #TODO: grab the dataset clean name and store it in the new object
        detailed_series_data['series_name'] = "TODO"

        detailed_series_data['series_raw_observations'] = json.dumps(json.loads(fr.series.observations(series_identifier))['observations'])

        # convert to a dataframe and clean it up
        intermediate_state_dataframe = pd.read_json(observations)
        intermediate_state_dataframe.columns = ['realtime_start', 'realtime_end', 'date', detailed_series_data["series_name"]]
    
        #TODO: calc min and max date here (looking across all the dataframes)
        #calc the min, compare to the global min
        #calc the max, compare to the global max

        #TODO can a python object handle different types like JS?
        detailed_series_data['series_dataframed'] = df_cleanup(intermediate_state_dataframe)

        outgoing_dataseries_list.push(detailed_series_data) 

    #now that we have looped thru the list, we begin transformations

    #create the earliest to latest timerange
    #pandas.timerange(beginning,end)
    
    # shift every series over to the new frame (expect lots of NaNs)
    # reindex each series according to the fill instructions
    # return the resulting main dataframe as json to the UI
    
    
def df_cleanup(dframe):
    dframe.drop(axis=1, columns=['realtime_start','realtime_end'], inplace=True)
    #convert date strings to proper datetimes
    dframe['date'] = pd.to_datetime(dframe['date'])
    dframe.set_index('date', inplace=True)
    #cleaner index in YYMMDD format
    dframe.index = dframe.index.strftime('%Y/%m/%d') 

    return dframe



#TODO should this function have a slot for params? or is it an API route?
def retrieve_raw_fred_data(series_identifier):
    # call Fred again....
    return json.dumps(json.loads(fr.series.observations(series_identifier)))




In [56]:
def object_2_dataframe(title, incoming_object):
    d_frame = pd.read_json(json.dumps(incoming_object))
    d_frame.columns = ['realtime_start', 'realtime_end', 'date', title]
    return d_frame
    
def df_cleanup(dframe, columns_to_remove=None):
    if columns_to_remove is not None:
        dframe.drop(axis=1, columns=columns_to_remove, inplace=True)
    
    #convert date strings to proper datetimes
    dframe['date'] = pd.to_datetime(dframe['date'])
    dframe.set_index('date', inplace=True)
    #cleaner index in YYMMDD format
    #dframe.index = dframe.index.strftime('%Y/%m/%d') 
    return dframe

def retrieve_raw_fred_data(series_identifier):
    # call Fred again....
    return json.loads(fr.series.observations(series_identifier))

def retrieve_series_name(series_identifier):
    return json.loads(fr.series.details(series_identifier))['seriess'][0]['title']

def retrieve_series_frequency(series_identifier):
    return json.loads(fr.series.details(series_identifier))['seriess'][0]['frequency_short']

def resample_hof(incoming_data_object, target_output_frequency):
    series_fill_methodology = incoming_data_object['series_fill_methodology']

    if series_fill_methodology == "fill":
        resampled_dataset = transform_fill(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "prorate":
        resampled_dataset = transform_prorate(incoming_data_object['series_dataframed'], target_output_frequency, incoming_data_object['series_frequency'])
    elif series_fill_methodology == "interpolate":
        resampled_dataset = transform_interpolate(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "average":
        resampled_dataset = transform_average(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "sum":
        resampled_dataset = transform_sum(incoming_data_object['series_dataframed'], target_output_frequency)
    else:
        resampled_dataset = incoming_data_object

    return resampled_dataset

##### Noah OG code
# fill
def transform_fill(series, transform_target):
    outputSeries = series.resample(transform_target).bfill()
    return outputSeries

# prorate
def transform_prorate(series, transform_target, series_base_freq):
    # Resample data to target frequency
    newDF = series
    if series_base_freq == "M":
        newDF["newdate"] = newDF.index + pd.offsets.MonthEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.MonthEnd(-1)
    elif series_base_freq == "Q":
        newDF["newdate"] = newDF.index + pd.offsets.QuarterEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.QuarterEnd(-1)
    elif series_base_freq == "A":
        newDF["newdate"] = newDF.index + pd.offsets.YearEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.YearEnd(-1)
    else:  # Assume week
        newDF["newdate"] = newDF.index
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.Week(-1)

    newDF["divisor"] = (newDF["newdate"] - newDF["prevdate"]).dt.days
    outputSeries = newDF.resample(transform_target).bfill()
    outputSeries["oldVal"] = pd.to_numeric(outputSeries.iloc[:, 0], errors="coerce")

    outputSeries.iloc[:, 0] = outputSeries["oldVal"] / outputSeries["divisor"]
    outputSeries.drop(
        axis=1, columns=["newdate", "divisor", "oldVal", "prevdate"], inplace=True
    )
    return outputSeries


# interpolate
def transform_interpolate(series, transform_target):
    outputSeries = series.resample(transform_target).interpolate()
    # Currently using linear interpolation, but may want to consider using a spline - good stats question
    return outputSeries


# average
def transform_average(series, transform_target):
    outputSeries = series.resample(transform_target).mean()
    return outputSeries


# sum
def transform_sum(series, transform_target):
    outputSeries = series.resample(transform_target).sum()
    return outputSeries

#receive payload (series names + fill prefs, target frequency)


#temp stub
requested_series_identifier_list = [
    {
        "series_identifier":"HOUST", 
        "fill_methodology":"interpolate"
        }, 
    {
        "series_identifier":"MORTGAGE30US", 
        "fill_methodology":"interpolate"
        }]

target_output_frequency = 'D'

#define list of objects to send to the next step
outgoing_dataseries_list = []
earliest_date = None
latest_date = None

#looooooooooooop
for requested_series in requested_series_identifier_list:

    #2 params on each series object
    series_identifier = requested_series['series_identifier']
    fill_methodology = requested_series['fill_methodology']
    

    #define outgoing data  #TODO: future state hold on to the name and description from the original search result
    detailed_series_data = {
        'series_name': 'placeholder',
        'series_identifier': series_identifier,
        'series_description': 'placeholder',
        'series_frequency': 'placeholder',
        'series_raw_api_response': 'placeholder',
        'series_raw_observations': 'placeholder',
        'series_dataframed': 'placeholder',
        'series_fill_methodology': fill_methodology
        }

    #get the raw data from FRED
    detailed_series_data['series_raw_api_response'] = retrieve_raw_fred_data(series_identifier)
    detailed_series_data['series_name'] = retrieve_series_name(series_identifier)
    detailed_series_data['series_frequency'] = retrieve_series_frequency(series_identifier)
    detailed_series_data['series_raw_observations'] = detailed_series_data['series_raw_api_response']['observations']

    # convert to a dataframe and clean it up
    intermediate_state_dataframe = object_2_dataframe(detailed_series_data['series_name'], detailed_series_data['series_raw_observations'])
    detailed_series_data['series_dataframed'] = df_cleanup(intermediate_state_dataframe, ['realtime_start','realtime_end'])
    
    #calc min and max date here (looking across all the dataframes)
    local_min  = detailed_series_data['series_dataframed'].index.min()
    local_max  = detailed_series_data['series_dataframed'].index.max()
    
    if earliest_date == None:
        earliest_date = local_min

    elif local_min < earliest_date:
        earliest_date = local_min

    if latest_date == None:
        latest_date = local_max

    elif local_max > latest_date:
        latest_date = local_max

    #add to list of objects    
    outgoing_dataseries_list.append(detailed_series_data)

#detailed_series_data['series_dataframed'] = detailed_series_data['series_dataframed'].resample(target_output_frequency).ffill()
    
#create the overarching earliest to latest timerange
main_frame = pd.DataFrame(np.nan, index=pd.date_range(earliest_date, latest_date), columns=['NaNs'])
main_frame.drop(axis=1, columns=['NaNs'], inplace=True)

# shift every series over to the new frame (expect lots of NaNs)
for dataseries_object in outgoing_dataseries_list:
    #resample & join the frame into main
    #TODO: figure out how to fill the NaNs that are created when the date range extends before or after the dataset
    main_frame = main_frame.join(resample_hof(dataseries_object, target_output_frequency))

# return the resulting main dataframe as json to the UI
main_frame.index = main_frame.index.strftime('%Y/%m/%d') 
print(main_frame.head())

            New Privately-Owned Housing Units Started: Total Units  \
1959/01/01                                        1657.000000        
1959/01/02                                        1657.322581        
1959/01/03                                        1657.645161        
1959/01/04                                        1657.967742        
1959/01/05                                        1658.290323        

            30-Year Fixed Rate Mortgage Average in the United States  
1959/01/01                                                NaN         
1959/01/02                                                NaN         
1959/01/03                                                NaN         
1959/01/04                                                NaN         
1959/01/05                                                NaN         


In [ ]:

#### TODO JT SUDOCODE ####
#### BEGIN

#receive payload ([series names + fill prefs], target frequency)

#read in the target frequency    
#expect data_series to be a list of objects
#looooooooooooop thru all series

    # get each series raw, create an object for each
    # once we have all objects, compute the max and min dates for the series
    # create a daily frame covering that date range

# shift every series over to the new frame (expect lots of NaNs)
# reindex each series according to the fill instructions
# return the resulting main dataframe as json to the UI

#### END

#we receive a payload with a set of series to retrieve, and the frequency we want to transform them into
# we pull the frequency from it (only 1 shared on both datasets)
# loop thru provided tickers
#'{"series":{"HOUST":"interpolate","MORTGAGE30US":"interpolate"},"outputFreq":"D"}'

#we then form the dictionary we'll be adding our datasets to
# API_SERIES_DICT = API_PAYLOAD["series"]
# results in API_SERIES_DICT = {"HOUST":"interpolate","MORTGAGE30US":"interpolate"}

# set the target output frequency for all datasets
    # then u loop thru each series in the dict
    #  grabbing the key from the dict
    # sending that ID to the fred retrieval function
    # also sending the fill method for that series (e.g. interpolate, sum, average)

# 3 vars get passed to produce the master output
    # desired data freuqnecy
    # name of each series
    # fill method for each series

# noah then retieves each dataset, and adjusts to end of period at retireval time
# e.g. end of month, quarter, year  
# TODO: do we want to make this it's own function? might be too assumptive

# TODO var naming is very confusing



In [6]:
#add search
params = {'limit':50,}
res = fr.series.search('money service index',params=params)
# print(res)
observations = json.dumps(json.loads(res))
SEARCH_RES_DF = pd.read_json(observations)
SEARCH_RES_DF_NEW = SEARCH_RES_DF['seriess']
# print(SEARCH_RES_DF_NEW.head())
# Convert back to JSON
RETURN_JSON_OBJECT = SEARCH_RES_DF_NEW.to_json()
print(RETURN_JSON_OBJECT)




{"0":{"id":"MSIM2","realtime_start":"2023-01-29","realtime_end":"2023-01-29","title":"Monetary Services Index: M2 (preferred)","observation_start":"1967-01-01","observation_end":"2013-12-01","frequency":"Monthly","frequency_short":"M","units":"Billions of Dollars","units_short":"Bil. of $","seasonal_adjustment":"Seasonally Adjusted","seasonal_adjustment_short":"SA","last_updated":"2014-01-17 07:16:44-06","popularity":25,"group_popularity":25,"notes":"The MSI measure the flow of monetary services received each period by households and firms from their holdings of monetary assets (levels of the indexes are sometimes referred to as Divisia monetary aggregates).\nPreferred benchmark rate equals 100 basis points plus the largest rate in the set of rates.\nAlternative benchmark rate equals the larger of the preferred benchmark rate and the Baa corporate bond yield.\nMore information about the new MSI can be found at\nhttp:\/\/research.stlouisfed.org\/msi\/index.html."},"1":{"id":"MSIMZMP","r

In [10]:
import json
from fred import Fred

def object_2_dataframe(title, incoming_object):
    d_frame = pd.read_json(json.dumps(incoming_object))
    d_frame.columns = ['realtime_start', 'realtime_end', 'date', title]
    return d_frame
    
def df_cleanup(dframe, columns_to_remove=None):
    if columns_to_remove is not None:
        dframe.drop(axis=1, columns=columns_to_remove, inplace=True)
    
    #convert date strings to proper datetimes
    dframe['date'] = pd.to_datetime(dframe['date'])
    dframe.set_index('date', inplace=True)
    #cleaner index in YYMMDD format
    #dframe.index = dframe.index.strftime('%Y/%m/%d') 
    return dframe

def retrieve_raw_fred_data(series_identifier):
    # call Fred again....
    return json.loads(fr.series.observations(series_identifier))


#TODO: merge these 2 functions into 1 that accepts different characteristic names
def retrieve_series_name(series_identifier):
    return json.loads(fr.series.details(series_identifier))['seriess'][0]['title']

def retrieve_series_frequency(series_identifier):
    return json.loads(fr.series.details(series_identifier))['seriess'][0]['frequency_short']

def resample_hof(incoming_data_object, target_output_frequency):
    series_fill_methodology = incoming_data_object['series_fill_methodology']

    if series_fill_methodology == "fill":
        resampled_dataset = transform_fill(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "prorate":
        resampled_dataset = transform_prorate(incoming_data_object['series_dataframed'], target_output_frequency, incoming_data_object['series_frequency'])
    elif series_fill_methodology == "interpolate":
        resampled_dataset = transform_interpolate(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "average":
        resampled_dataset = transform_average(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "sum":
        resampled_dataset = transform_sum(incoming_data_object['series_dataframed'], target_output_frequency)
    else:
        resampled_dataset = incoming_data_object

    return resampled_dataset

##### Noah OG code
# fill
def transform_fill(series, transform_target):
    outputSeries = series.resample(transform_target).bfill()
    return outputSeries

# prorate
def transform_prorate(series, transform_target, series_base_freq):
    # Resample data to target frequency
    newDF = series
    if series_base_freq == "M":
        newDF["newdate"] = newDF.index + pd.offsets.MonthEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.MonthEnd(-1)
    elif series_base_freq == "Q":
        newDF["newdate"] = newDF.index + pd.offsets.QuarterEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.QuarterEnd(-1)
    elif series_base_freq == "A":
        newDF["newdate"] = newDF.index + pd.offsets.YearEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.YearEnd(-1)
    else:  # Assume week
        newDF["newdate"] = newDF.index
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.Week(-1)

    newDF["divisor"] = (newDF["newdate"] - newDF["prevdate"]).dt.days
    outputSeries = newDF.resample(transform_target).bfill()
    outputSeries["oldVal"] = pd.to_numeric(outputSeries.iloc[:, 0], errors="coerce")

    outputSeries.iloc[:, 0] = outputSeries["oldVal"] / outputSeries["divisor"]
    outputSeries.drop(
        axis=1, columns=["newdate", "divisor", "oldVal", "prevdate"], inplace=True
    )
    return outputSeries


# interpolate
def transform_interpolate(series, transform_target):
    outputSeries = series.resample(transform_target).interpolate()
    # Currently using linear interpolation, but may want to consider using a spline - good stats question
    return outputSeries


# average
def transform_average(series, transform_target):
    outputSeries = series.resample(transform_target).mean()
    return outputSeries


# sum
def transform_sum(series, transform_target):
    outputSeries = series.resample(transform_target).sum()
    return outputSeries

#read in the target frequency    
target_output_frequency = 'D'

#expect data_series to be a list of objects
requested_series_identifier_list  = [{"series_identifier":"CPHPTT01EZM659N", "fill_methodology":"interpolate"}, {"series_identifier":"EXHOSLUSM495S", "fill_methodology":"interpolate"}]

#get the raw data from FRED

print(object_2_dataframe('CPHPTT01EZM659N', retrieve_raw_fred_data('CPHPTT01EZM659N')['observations']).head())
print(object_2_dataframe('EXHOSLUSM495S', retrieve_raw_fred_data('EXHOSLUSM495S')['observations']).head())
print(object_2_dataframe('HOUST', retrieve_raw_fred_data('HOUST')['observations']).head())
print(object_2_dataframe('MORTGAGE30US', retrieve_raw_fred_data('MORTGAGE30US')['observations']).head())





  realtime_start realtime_end       date  CPHPTT01EZM659N
0     2023-02-12   2023-02-12 1997-01-01              2.2
1     2023-02-12   2023-02-12 1997-02-01              2.0
2     2023-02-12   2023-02-12 1997-03-01              1.7
3     2023-02-12   2023-02-12 1997-04-01              1.5
4     2023-02-12   2023-02-12 1997-05-01              1.5
  realtime_start realtime_end       date EXHOSLUSM495S
0     2023-02-12   2023-02-12 2021-11-01             .
1     2023-02-12   2023-02-12 2021-12-01     6090000.0
2     2023-02-12   2023-02-12 2022-01-01     6490000.0
3     2023-02-12   2023-02-12 2022-02-01     5930000.0
4     2023-02-12   2023-02-12 2022-03-01     5750000.0
  realtime_start realtime_end       date  HOUST
0     2023-02-12   2023-02-12 1959-01-01   1657
1     2023-02-12   2023-02-12 1959-02-01   1667
2     2023-02-12   2023-02-12 1959-03-01   1620
3     2023-02-12   2023-02-12 1959-04-01   1590
4     2023-02-12   2023-02-12 1959-05-01   1498
  realtime_start realtime_end     

In [11]:
# convert to a dataframe and clean it up
intermediate_state_dataframe = object_2_dataframe(detailed_series_data['series_name'], detailed_series_data['series_raw_observations'])
detailed_series_data['series_dataframed'] = df_cleanup(intermediate_state_dataframe, ['realtime_start','realtime_end'])

#calc min and max date here (looking across all the dataframes)
local_min  = detailed_series_data['series_dataframed'].index.min()
local_max  = detailed_series_data['series_dataframed'].index.max()

if earliest_date == None:
    earliest_date = local_min

elif local_min < earliest_date:
    earliest_date = local_min

if latest_date == None:
    latest_date = local_max

elif local_max > latest_date:
    latest_date = local_max

#add to list of objects    
outgoing_dataseries_list.append(detailed_series_data)
outgoing_df_list.append(detailed_series_data['series_dataframed'])

#detailed_series_data['series_dataframed'] = detailed_series_data['series_dataframed'].resample(target_output_frequency).ffill()

#create the overarching earliest to latest timerange
main_frame = pd.DataFrame(np.nan, index=pd.date_range(earliest_date, latest_date), columns=['NaNs'])
main_frame.drop(axis=1, columns=['NaNs'], inplace=True)

# shift every series over to the new frame (expect lots of NaNs)
for dataseries_object in outgoing_dataseries_list:
#resample & join the frame into main
#TODO: figure out how to fill the NaNs that are created when the date range extends before or after the dataset
main_frame = main_frame.join(resample_hof(dataseries_object, target_output_frequency))

# return the resulting main dataframe as json to the UI
main_frame.index = main_frame.index.strftime('%Y/%m/%d') 

IndentationError: unexpected indent (1806503431.py, line 2)

In [18]:

from datetime import date
import time
import os
import json
from fred import Fred
import pandas as pd
import numpy as np

#INIT THE CLIENT
API_KEY = os.environ.get('FRED_API_KEY')
fr = Fred(api_key=API_KEY,response_type='json')

def data_request_hof(target_output_frequency, requested_series_identifier_list):

    #define list of objects to send to the next step
    outgoing_dataseries_list = []
    outgoing_df_list = []
    earliest_date = None
    latest_date = None

    #looooooooooooop
    for requested_series in requested_series_identifier_list:

        #2 params on each series object
        series_identifier = requested_series['series_identifier']
        fill_methodology = requested_series['fill_methodology']

        #define outgoing data  #TODO: future state hold on to the name and description from the original search result
        detailed_series_data = {
            'series_name': 'placeholder',
            'series_identifier': series_identifier,
            'series_description': 'placeholder',
            'series_frequency': 'placeholder',
            'series_raw_api_response': 'placeholder',
            'series_raw_observations': 'placeholder',
            'series_dataframed': 'placeholder',
            'series_fill_methodology': fill_methodology
            }

        #get the raw data from FRED
        detailed_series_data['series_raw_api_response'] = retrieve_raw_fred_data(series_identifier)
        detailed_series_data['series_name'] = retrieve_series_name(series_identifier)
        detailed_series_data['series_frequency'] = retrieve_series_frequency(series_identifier)
        detailed_series_data['series_raw_observations'] = detailed_series_data['series_raw_api_response']['observations']

        # convert to a dataframe and clean it up
        intermediate_state_dataframe = object_2_dataframe(detailed_series_data['series_name'], detailed_series_data['series_raw_observations'])
        detailed_series_data['series_dataframed'] = df_cleanup(intermediate_state_dataframe, ['realtime_start','realtime_end'])
        
        #calc min and max date here (looking across all the dataframes)
        local_min  = detailed_series_data['series_dataframed'].index.min()
        local_max  = detailed_series_data['series_dataframed'].index.max()
        
        if earliest_date == None:
            earliest_date = local_min

        elif local_min < earliest_date:
            earliest_date = local_min

        if latest_date == None:
            latest_date = local_max

        elif local_max > latest_date:
            latest_date = local_max

        #add to list of objects    
        outgoing_dataseries_list.append(detailed_series_data)
        outgoing_df_list.append(detailed_series_data['series_dataframed'])

    #detailed_series_data['series_dataframed'] = detailed_series_data['series_dataframed'].resample(target_output_frequency).ffill()
        
    #create the overarching earliest to latest timerange
    main_frame = pd.DataFrame(np.nan, index=pd.date_range(earliest_date, latest_date), columns=['NaNs'])
    main_frame.drop(axis=1, columns=['NaNs'], inplace=True)

    # shift every series over to the new frame (expect lots of NaNs)
    for dataseries_object in outgoing_dataseries_list:
        #resample & join the frame into main
        #TODO: figure out how to fill the NaNs that are created when the date range extends before or after the dataset
        main_frame = main_frame.join(resample_hof(dataseries_object, target_output_frequency))

    # return the resulting main dataframe as json to the UI
    main_frame.index = main_frame.index.strftime('%Y/%m/%d') 
    
    return main_frame.to_json()
    

def object_2_dataframe(title, incoming_object):
    d_frame = pd.read_json(json.dumps(incoming_object))
    d_frame.columns = ['realtime_start', 'realtime_end', 'date', title]
    print(d_frame[title].dtype)

    #check if data came in as a objects, if so, convert it
    if d_frame[title].dtype not in ['float64', 'int64']:
         d_frame[title] = d_frame[title].astype(str).astype(int)

    return d_frame
    
def df_cleanup(dframe, columns_to_remove=None):
    if columns_to_remove is not None:
        dframe.drop(axis=1, columns=columns_to_remove, inplace=True)
    
    #convert date strings to proper datetimes
    dframe['date'] = pd.to_datetime(dframe['date'])
    dframe.set_index('date', inplace=True)

    
    return dframe

def retrieve_raw_fred_data(series_identifier):
    # call Fred again....
    return json.loads(fr.series.observations(series_identifier))


#TODO: merge these 2 functions into 1 that accepts different characteristic names
def retrieve_series_name(series_identifier):
    return json.loads(fr.series.details(series_identifier))['seriess'][0]['title']

def retrieve_series_frequency(series_identifier):
    return json.loads(fr.series.details(series_identifier))['seriess'][0]['frequency_short']

def resample_hof(incoming_data_object, target_output_frequency):
    series_fill_methodology = incoming_data_object['series_fill_methodology']

    if series_fill_methodology == "fill":
        resampled_dataset = transform_fill(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "prorate":
        resampled_dataset = transform_prorate(incoming_data_object['series_dataframed'], target_output_frequency, incoming_data_object['series_frequency'])
    elif series_fill_methodology == "interpolate":
        resampled_dataset = transform_interpolate(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "average":
        resampled_dataset = transform_average(incoming_data_object['series_dataframed'], target_output_frequency)
    elif series_fill_methodology == "sum":
        resampled_dataset = transform_sum(incoming_data_object['series_dataframed'], target_output_frequency)
    else:
        resampled_dataset = incoming_data_object

    return resampled_dataset

##### Noah OG code
# fill
def transform_fill(series, transform_target):
    outputSeries = series.resample(transform_target).bfill()
    return outputSeries

# prorate
def transform_prorate(series, transform_target, series_base_freq):
    # Resample data to target frequency
    newDF = series
    if series_base_freq == "M":
        newDF["newdate"] = newDF.index + pd.offsets.MonthEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.MonthEnd(-1)
    elif series_base_freq == "Q":
        newDF["newdate"] = newDF.index + pd.offsets.QuarterEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.QuarterEnd(-1)
    elif series_base_freq == "A":
        newDF["newdate"] = newDF.index + pd.offsets.YearEnd(0)
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.YearEnd(-1)
    else:  # Assume week
        newDF["newdate"] = newDF.index
        newDF["prevdate"] = newDF.newdate.shift(1)
        newDF.iloc[0, newDF.columns.get_loc("prevdate")] = newDF.iloc[
            0, newDF.columns.get_loc("newdate")
        ] + pd.offsets.Week(-1)

    newDF["divisor"] = (newDF["newdate"] - newDF["prevdate"]).dt.days
    outputSeries = newDF.resample(transform_target).bfill()
    outputSeries["oldVal"] = pd.to_numeric(outputSeries.iloc[:, 0], errors="coerce")

    outputSeries.iloc[:, 0] = outputSeries["oldVal"] / outputSeries["divisor"]
    outputSeries.drop(
        axis=1, columns=["newdate", "divisor", "oldVal", "prevdate"], inplace=True
    )
    return outputSeries


# interpolate
def transform_interpolate(series, transform_target):
    outputSeries = series.resample(transform_target).interpolate()
    # Currently using linear interpolation, but may want to consider using a spline - good stats question
    return outputSeries


# average
def transform_average(series, transform_target):
    outputSeries = series.resample(transform_target).mean()
    return outputSeries


# sum
def transform_sum(series, transform_target):
    outputSeries = series.resample(transform_target).sum()
    return outputSeries

print(data_request_hof('D', [{"series_identifier":"CPHPTT01EZM659N", "fill_methodology":"interpolate"}, {"series_identifier":"EXHOSLUSM495S", "fill_methodology":"interpolate"}, {"series_identifier":"HOUST", "fill_methodology":"interpolate"}, {"series_identifier":"MORTGAGE30US", "fill_methodology":"interpolate"}]))

float64
object
int64
float64


TypeError: Cannot interpolate with all object-dtype columns in the DataFrame. Try setting at least one column to a numeric dtype.